In [1]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv("data/DataSample.csv")
df.dtypes

_ID            int64
 TimeSt       object
Country       object
Province      object
City          object
Latitude     float64
Longitude    float64
dtype: object

Notice how TimeSt column name has an empty space in its name.

In [2]:
df.rename(columns={" TimeSt": "TimeSt"}, inplace=True)
df.dtypes

_ID            int64
TimeSt        object
Country       object
Province      object
City          object
Latitude     float64
Longitude    float64
dtype: object

Looks better now.

In [3]:
df

,_ID,TimeSt,Country,Province,City,Latitude,Longitude
0,4516516,2017-06-21 00:00:00.143,CA,ON,Waterloo,43.49347,-80.49123
1,4516547,2017-06-21 18:00:00.193,CA,ON,London,42.93990,-81.27090
2,4516550,2017-06-21 15:00:00.287,CA,ON,Guelph,43.57760,-80.22010
3,4516600,2017-06-21 15:00:00.307,CA,ON,Stratford,43.37160,-80.97730
4,4516613,2017-06-21 15:00:00.497,CA,ON,Stratford,43.37160,-80.97730
...,...,...,...,...,...,...,...
22020,5614801,2017-06-21 12:23:07.880,CA,ON,Saint Catharines,43.16440,-79.24560
22021,5614888,2017-06-21 08:23:01.793,CA,AB,Calgary,51.02093,-114.10621
22022,5614909,2017-06-21 00:23:07.903,CA,ON,Whitby,43.88730,-78.94220
22023,5614912,2017-06-21 11:23:07.953,CA,ON,Oakville,43.49340,-79.71260


1. Cleanup:
Find the sample dataset of request logs in `data/DataSample.csv`. 
We consider records with identical `geoinfo` and `timest` as suspicious. 
Please clean up the sample dataset by filtering out those questionable request records.

I am assuming that "geoinfo", mentioned above, is a combination of "Latitude" and "Longitude" data. Hence, I will consider the following subset for the duplicate check: "TimeSt","Latitude","Longitude".

drop_duplicates() below considers duplicates where:  "TimeSt"_n == "TimeSt"_m AND "Latitude"_n == "Latitude"_m AND "Longitude"_n == "Longitude"_m

In [4]:
df.drop_duplicates(subset =["TimeSt","Latitude","Longitude"],keep=False,inplace=True,ignore_index=True)
#keep=False --> drop all instances of identical data
#if we want to keep one record, then keep="first"


df

,_ID,TimeSt,Country,Province,City,Latitude,Longitude
0,4516516,2017-06-21 00:00:00.143,CA,ON,Waterloo,43.49347,-80.49123
1,4516547,2017-06-21 18:00:00.193,CA,ON,London,42.93990,-81.27090
2,4516550,2017-06-21 15:00:00.287,CA,ON,Guelph,43.57760,-80.22010
3,4516600,2017-06-21 15:00:00.307,CA,ON,Stratford,43.37160,-80.97730
4,4516613,2017-06-21 15:00:00.497,CA,ON,Stratford,43.37160,-80.97730
...,...,...,...,...,...,...,...
17968,5614689,2017-06-21 13:23:01.370,CA,ON,London,42.96110,-81.24310
17969,5614801,2017-06-21 12:23:07.880,CA,ON,Saint Catharines,43.16440,-79.24560
17970,5614909,2017-06-21 00:23:07.903,CA,ON,Whitby,43.88730,-78.94220
17971,5614912,2017-06-21 11:23:07.953,CA,ON,Oakville,43.49340,-79.71260


At this point, the dataframe should be free of records with identical geoinfo and timest.

2. Label: Assign each *request* (from `data/DataSample.csv`) to the closest (i.e., minimum distance) *POI* (from `data/POIList.csv`). Note: a *POI* is a geographical Point of Interest.

In [5]:
df2 = pd.read_csv("data/POIList.csv")
df2.dtypes

POIID         object
 Latitude    float64
Longitude    float64
dtype: object

Renaming " Latitude" to "Latitude". 

In [6]:
df2.rename(columns={" Latitude": "Latitude"}, inplace=True)
df2.dtypes

POIID         object
Latitude     float64
Longitude    float64
dtype: object

In [7]:
df2

,POIID,Latitude,Longitude
0,POI1,53.546167,-113.485734
1,POI2,53.546167,-113.485734
2,POI3,45.521629,-73.566024
3,POI4,45.224830,-63.232729


POI1 and POI2 are identical, so one of the POIs is redundant.

In [8]:
df2.drop_duplicates(subset=["Latitude","Longitude"],keep="first",inplace=True,ignore_index=True)
df2

,POIID,Latitude,Longitude
0,POI1,53.546167,-113.485734
1,POI3,45.521629,-73.566024
2,POI4,45.224830,-63.232729


In [9]:
df["POIID"] = "NA";
xy = ['Latitude', 'Longitude']

for i in range(0,df.shape[0]):
    distance_array = np.zeros(shape=(df2.shape[0]));
    for x in range(0,df2.shape[0]):
         distance_array[x] = np.sum((df[i:i+1][xy].values - df2[x:x+1][xy].values)**2, axis=1);
    df.loc[i,"POIID"] = df2["POIID"][distance_array.argmin()]
 


In [10]:
df

,_ID,TimeSt,Country,Province,City,Latitude,Longitude,POIID
0,4516516,2017-06-21 00:00:00.143,CA,ON,Waterloo,43.49347,-80.49123,POI3
1,4516547,2017-06-21 18:00:00.193,CA,ON,London,42.93990,-81.27090,POI3
2,4516550,2017-06-21 15:00:00.287,CA,ON,Guelph,43.57760,-80.22010,POI3
3,4516600,2017-06-21 15:00:00.307,CA,ON,Stratford,43.37160,-80.97730,POI3
4,4516613,2017-06-21 15:00:00.497,CA,ON,Stratford,43.37160,-80.97730,POI3
...,...,...,...,...,...,...,...,...
17968,5614689,2017-06-21 13:23:01.370,CA,ON,London,42.96110,-81.24310,POI3
17969,5614801,2017-06-21 12:23:07.880,CA,ON,Saint Catharines,43.16440,-79.24560,POI3
17970,5614909,2017-06-21 00:23:07.903,CA,ON,Whitby,43.88730,-78.94220,POI3
17971,5614912,2017-06-21 11:23:07.953,CA,ON,Oakville,43.49340,-79.71260,POI3


Each request (from data/DataSample.csv) is assigned to the closest POI (from data/POIList.csv). 
Note: POI2 is excluded from matching because it is identical to POI1.

In [11]:
POI1 = df[df["POIID"] == "POI1"].count()[0]
POI3 = df[df["POIID"] == "POI3"].count()[0]
POI4 = df[df["POIID"] == "POI4"].count()[0]

print(f"""Locations matched to POI1: {POI1} \nLocations matched to POI3: {POI3} \nLocations matched to POI4: {POI4}""")

Locations matched to POI1: 8725 
Locations matched to POI3: 8821 
Locations matched to POI4: 427
